In [211]:
from datasets import load_dataset, load_from_disk
import wordninja
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import re
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import datasets
import transformers
from transformers import AutoTokenizer, default_data_collator, get_scheduler
from hf_transformers.src.transformers.models.bert.configuration_bert import BertConfig
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForPreTraining

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

import sys
import math

import collections


from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


import openai

# Create your views here.
openai.api_key='YOUR OPENAI API KEY'

In [ ]:
#WARNING
#cacheディレクトリの指定は絶対マウント先のフォルダにするように
#そうしないとdockerイメージを管理している研究室サーバーの/がパンパンになってしまう
#/が容量オーバーすると何も動かなくなって他の人に迷惑
!export HF_DATASETS_CACHE="/home/"

In [4]:
df_summary = pd.read_csv("./logdata/log_summary_pairs.csv", usecols=["input", "summary"])

In [165]:
max(list(df_summary["summary"]), key=lambda x : len(x))

420

In [190]:
import openai

# Create your views here.
openai.api_key='YOUR OPENAI API KEY'

def ask_GPT(input_log, max_tokens=64):
    prompt = make_prompt(input_log)
    return GPT(prompt, max_tokens=max_tokens)
    
#プロンプト生成
def make_prompt(input_log, num_samples=3):
    prompt = """Could you briefly summarize a log message like the following examples? Note: Please do NOT include any numbers and special characters in your answer."""

    samples = df_summary.sample(num_samples)

    i = 0
    for log, summary in zip(samples["input"], samples["summary"]):
        prompt += f"\nQ{i + 1}: {log}\nA{i + 1}: {summary}\n"
        i += 1
        
    prompt += f"\nQ{i + 1}:{input_log}\n"

    return prompt

#chatGPTからプロンプト(user_query)対する答えを得る関数
def GPT(user_query, model_engine="text-davinci-003", max_tokens=64):
    '''
    This function uses the OpenAI API to generate a response to the given
    user_query using the ChatGPT model
    '''
    # Use the OpenAI API to generate a response
    completion = openai.Completion.create(
          engine = model_engine,
          prompt = user_query,
          max_tokens = max_tokens,
          n = 1,
          temperature = 1.0,
    )
    response = completion.choices[0].text
    return response

In [127]:
sample_input

'INFO dfs.FSNamesystem : BLOCK* NameSystem.addStoredBlock : blockMap updated : 10.251.202.134 : 50010 is added to blk_-6250047579414065511 size 67108864'

In [152]:
df_test = pd.read_csv("./logdata/test_summary.csv", usecols=["input", "summary"])
df_test

,input,summary
0,switch module error link in reset,link in reset link ok link error
1,info df s data node data x ce iver served bloc...,verification succeeded got exception served block
2,g it hub com close bytes sent bytes kb receive...,open through proxy bytes sent bytes received c...
3,tcp conn tencent com error could not connect t...,open through proxy bytes sent bytes received c...
4,node status not responding,node status not responding node status configu...
...,...,...
1193,info client attempting to renew session x f a at,connection request will be dropped client atte...
1194,node status running,node status running
1195,warn connection broken for id my id error,received connection request connection broken ...
1196,switch module error link ok,link in reset link ok link error


In [191]:
print(make_prompt(sample_input))

Could you briefly summarize a log message like the following examples? Note: Please do NOT include any numbers and special characters in your answer.
Q1: INFO dfs.FSNamesystem : BLOCK* NameSystem.addStoredBlock : blockMap updated : 10.250.15.67 : 50010 is added to blk_6776004133998486732 size 67108864
A1: PacketResponder terminating; Received block; blockMap updated; Receiving block;

Q2: - proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
A2: open through proxy; bytes sent; bytes received; close;

Q3: RAS APP FATAL ciod : Error reading message prefix on CioStream socket to 172.16.96.116 : 47034 , Link has been severed
A3: Received signal; Error reading message prefix;

Q4:INFO dfs.FSNamesystem : BLOCK* NameSystem.addStoredBlock : blockMap updated : 10.251.202.134 : 50010 is added to blk_-6250047579414065511 size 67108864



In [195]:
df_summary = pd.read_csv("./logdata/test_summary.csv", usecols=["input", "summary"])

In [201]:
df_summary

In [200]:
recalls = []
precisions = []
f_scores = []

for i in tqdm(range(len(df_summary))):
    sample_input = df_summary.iloc[i]["input"]
    sample_ans = df_summary.iloc[i]["summary"]
    ans = ask_GPT(sample_input, max_tokens=64)
    rouge_score = rouge.get_scores(ans, sample_ans)[0]
    recalls.append(rouge_score["rouge-1"]["r"])
    precisions.append(rouge_score["rouge-1"]["p"])
    f_scores.append(rouge_score["rouge-1"]["f"])

print(f"Recall: {sum(recalls)/len(df_summary)}, Precision: {sum(precisions)/len(df_summary)}, Rouge-1 F {sum(f_scores)/len(df_summary)}")

100%|██████████| 1198/1198 [1:43:40<00:00,  5.19s/it]

Recall: 0.44769360015051407, Precision: 0.3619654803530013, Rouge-1 F 0.3714160554722489


In [124]:
rouge = datasets.load_metric("rouge")

def compute_metrics(pred_str, label_str):
    # all unnecessary tokens are removed

    padding_length = max(len(pred_str), len(label_str)) - min(len(pred_str), len(label_str))
    
    if max(len(pred_str), len(label_str)) == len(pred_str):
        rouge_output = rouge.compute(predictions=pred_str, references=label_str+" "*(padding_length), rouge_types=["rouge1"])["rouge1"].mid
    else:
        rouge_output = rouge.compute(predictions=pred_str+" "*(padding_length), references=label_str, rouge_types=["rouge1"])["rouge1"].mid

    return {
        "rouge1_precision": round(rouge_output.precision, 4),
        "rouge1_recall": round(rouge_output.recall, 4),
        "rouge1_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [193]:
from rouge import Rouge 

rouge = Rouge()
rouge.get_scores(ans, sample_ans)

[{'rouge-1': {'r': 0.25, 'p': 0.13333333333333333, 'f': 0.173913038941399},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.25, 'p': 0.13333333333333333, 'f': 0.173913038941399}}]

In [204]:
ans = 

In [209]:
sample_input = df_summary.iloc[10]["input"]
sample_ans = df_summary.iloc[10]["summary"]

In [212]:
print(make_prompt(sample_input))
print("-"*100)
print(ask_GPT(sample_input, max_tokens=64))

Could you briefly summarize a log message like the following examples? Note: Please do NOT include any numbers and special characters in your answer.
Q1: ras kernel fatal data store interrupt caused by ic bi
A1: program interrupt data store interrupt

Q2: switch module fan fan speeds
A2: fan speeds

Q3: warn send worker leaving thread
A3: send worker received connection connection broken received connection interrupting send worker interrupted

Q4:info storage memory store block broadcast piece stored as bytes in memory estimated size mb free mb

----------------------------------------------------------------------------------------------------
A4: block stored in memory estimated size amount of space being used amount of space still available storage capacity data stored as bytes information about how much is being used/free
